In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ast
import re
import os

In [15]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics import confusion_matrix

In [16]:
df=pd.read_csv('CSV_TMP/BBD75.csv', sep=',',dtype='object', usecols = [3,5])
#df.head(1000)

In [17]:
def get_title(index):
    return df[df.index == index]["Titre"].values[0]
def get_index(title):
    return df[df.Titre == title]["index"].values[0]
df['index'] = [i for i in range(0, len(df))]

In [18]:
indexNames = df[( df['Titre'] == '0')].index
# Delete these row indexes from dataFrame
df.drop(indexNames , inplace=True)
indexNames = df[( df['Titre'] == 0)].index
# Delete these row indexes from dataFrame
df.drop(indexNames , inplace=True)

In [19]:
def replace(texte):
    new_string=texte.replace("\\t","")
    new_string=new_string.replace("\\n","")
    new_string=new_string.replace("<br/>","")
    new_string=new_string.replace("/p","")
    new_string=new_string.replace('<p class="footer" style="clear:both;border:0px;">','')
    new_string=new_string.replace("<>","")
    return new_string

In [20]:
df['Resume']=df['Resume'].apply(replace)

In [21]:
df.head(20)

,Titre,Resume,index
0,"Les Aventures de Tintin, tome 03 : Tintin en A...","['Dans ""Tintin en Amérique"" (1932), le héros c...",0
1,"Les Aventures de Tintin, tome 11 : Le Secret d...","[""Dans cette aventure, Tintin accompagne le Ca...",1
2,"Les Aventures de Tintin, tome 17 : On a marché...","['La fusée a décollé de Syldavie, emportant Ti...",2
3,"Les aventures de Tintin, Tome 6 : L'Oreille ca...","[""Afin de récupérer un fétiche volé, Tintin s'...",3
4,"Les aventures de Tintin, tome 16 : Objectif lune","['De retour de voyage, Tintin et le capitaine ...",4
5,"Les Aventures de Tintin, tome 12 : Le Trésor d...",['Dans Le Secret de la Licorne (1943) et Le Tr...,5
6,"Les Aventures de Tintin, tome 04 : Les Cigares...","[""Alors qu'il est en croisière sur un paquebot...",6
7,"Les Aventures de Tintin, tome 07 : L'Île Noire","['Un mystérieux avion met Tintin, pour la prem...",7
8,"Les Aventures de Tintin, tome 05 : Le Lotus bleu","[""Poursuivant jusqu'en Chine les trafiquants r...",8
9,"Astérix, tome 20 : Astérix en Corse","[""Nous sommes en 50 avant Jésus-Christ ; toute...",9


In [22]:
from sentence_transformers import SentenceTransformer
bert = SentenceTransformer('bert-base-nli-mean-tokens')

In [23]:
def decoupage(df,nbligne):
    ### df: Le Dataframe a découpé, nbligne: le nombre de ligne par document
    longueur_df=len(df)
    # Calcul du nombre de fichiers nécessaire
    nb_fichier=int(len(df)/nbligne) # Nombre de fichier avec nbligne
    reste= len(df)%nbligne # Et un document evec reste ligne
    idx=0
    while idx < nb_fichier:
        dftmp=df[idx*nbligne:(idx+1)*nbligne]
        dftmp.to_csv("CSV_TMP/RESUME/df"+"0"*(len(str(nb_fichier))-len(str(idx)))+str(idx)+".csv", sep="\t")
        idx+=1
    if reste!=0:
        dftmp=df[:-reste]
        dftmp.to_csv("CSV_TMP/RESUME/df"+str(idx)+".csv", sep="\t")
        
    return nb_fichier, reste

In [24]:
print(decoupage(df,1))

(11084, 0)


In [25]:
from os import listdir
path_to_dir="CSV_TMP/RESUME/"
def find_csv_filenames( path_to_dir, suffix=".csv" ):
    filenames = listdir(path_to_dir)
    return sorted([ filename for filename in filenames if filename.endswith( suffix ) ])

In [26]:
ListeCSVTEMP=find_csv_filenames("CSV_TMP/RESUME/")

In [68]:
def faireNLP(repertoire):
    ListeCSVTEMP=find_csv_filenames(repertoire)
    nb_fichier_traite=1
    for fichier in ListeCSVTEMP:
        dftmp=pd.read_csv(repertoire+fichier, sep='\t',dtype='object')
        dftmp['NLP']=[bert.encode(dftmp['Resume'].tolist())] 
        dftmp['NLP']=replace(dftmp['NLP'])
        if (nb_fichier_traite==0) or (nb_fichier_traite%10==0):
            print("Traitement de ",str((nb_fichier_traite/len(ListeCSVTEMP))*100), " %.")
        nb_fichier_traite+=1
        dftmp.to_csv(repertoire+fichier, sep='\t')

In [69]:
faireNLP("CSV_TMP/RESUME/")

Traitement de  16.39344262295082  %.
Traitement de  32.78688524590164  %.
Traitement de  49.18032786885246  %.
Traitement de  65.57377049180327  %.
Traitement de  81.9672131147541  %.
Traitement de  98.36065573770492  %.


In [79]:
df=pd.read_csv('CSV_TMP/RESUME/df00001.csv',sep='\t',usecols=[7])
df.head(1)


,NLP
0,[[-7.64782906e-01 8.15230131e-01 5.14360249e...


In [89]:
dftmp=[]
def prepare_liste(repertoire):
    ListeCSVTEMP=find_csv_filenames(repertoire)
    for fichier in ListeCSVTEMP:
          [dftmp["NLP"] for filename in pd.read_csv(repertoire+fichier, sep='\t',usecols = [5])]
    return dftmp

In [88]:
similarity =cosine_similarity(prepare_liste("CSV_TMP/RESUME/"))

TypeError: list indices must be integers or slices, not str

In [13]:
Lst4Similarity=prepare_liste("CSV_TMP/RESUME/")

NameError: name 'fichier' is not defined

In [195]:
print(Lst4Similarity)

[0    [[-3.57037097e-01  6.74990356e-01  9.63729739e...
Name: NLP, dtype: object]


In [196]:
print(Lst4Similarity)

[0    [[-3.57037097e-01  6.74990356e-01  9.63729739e...
Name: NLP, dtype: object]


In [ ]:
# Test en mode console
notOver = True
while(notOver):
    user_bd = input("Enter le titre de la BD que vous aimez : ")
# Generate Recommendations
    recommendations = sorted(list(enumerate(similarity[get_index(user_bd)])), key = lambda x:x[1], reverse = True)
    print("Les 3 recommandations que nous pouvons vous faire pour " + user_bd +  " sont: ")
    print(get_title(recommendations[1][0]), get_title(recommendations[2][0]), get_title(recommendations[3][0]), sep = "\n")
    decision = input("Entrer '0' pour sortir et ce que vous voulez pour continuer")
    if(int(decision) == 0):
        print("Bye")
        notOver = False